# First Homework (Rock-Paper-Scissors)

In [1]:
!pip install -q -U kaggle_environments

In [2]:
from kaggle_environments import evaluate

In [3]:
# Для различных цветов и жирного текста при выводе

class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [4]:
import random
import math


def get_score(left_move, right_move):
    # This method exists in this file so it can be consumed from rps.py and agents.py without a circular dependency
    delta = (
        right_move - left_move
        if (left_move + right_move) % 2 == 0
        else left_move - right_move
    )
    return 0 if delta == 0 else math.copysign(1, delta)


#0 - камень
#1 - бумага
#2 - ножницы


# 1.Всегда камень
def always_rock(observation, configuration):
    return 0


# 2.Всегда бумага
def always_paper(observation, configuration):
    return 1


# 3.Всегда ножницы
def always_scissors(observation, configuration):
    return 2


# 4.Копирование значения оппонента
def copy_opponent(observation, configuration):
    if observation.step > 0:
        return observation.lastOpponentAction
    else:
        return random.randrange(0, configuration.signs)


# 5.Либо бумага, либо ножницы. Камень никогда не использовать.
def never_rock(observation, configuration):
    return random.randrange(1, configuration.signs)


# 6.Либо камень, либо ножницы. Бумагу никогда не использовать.
def never_paper(observation, configuration):
    return random.choice([0, 2])


# 7.Либо камень, либо бумага. Ножницы никогда не использовать.
def never_scissors(observation, configuration):
    return random.randrange(0, 2)


# 8.Если в прошлом раунде програл, то меняем на значение, которое выиграет.
last_react_action = None

def react_if_lose(observation, configuration):
    global last_react_action
    if observation.step == 0:
        last_react_action = random.randrange(0, configuration.signs)
    elif get_score(last_react_action, observation.lastOpponentAction) <= 1:
        last_react_action = (observation.lastOpponentAction + 1) % (configuration.signs)

    return last_react_action


# 9.После каждого третьего раунда меняет на другое значение.
last_react_action_2 = random.randrange(0, 3)
def every_third_round(observation, configuration):
    global last_react_action_2
    if observation.step % configuration.signs == 0:
        last_react_action_2 = random.randrange(0, configuration.signs)
    return last_react_action_2


# 10.Начинает играть с камня, а дальше случайным образом выбирается значение.
def start_with_rock(observation, configuration):
    if observation.step == 0:
        return 0
    else:
        return random.randrange(0, configuration.signs)


# 11.Ходит камнем по два раза
last_react_action_3 = None
var_array = []
def move_with_a_rock_two_times(observation, configuration):
    global var_array
    global last_react_action_3
    
    # Первый ход
    if observation.step == 0:
        last_react_action_3 = random.randrange(0, configuration.signs)
        if last_react_action_3 == 0:
            var_array.extend([0, 0])
        else:
            var_array.append(last_react_action_3)
            last_react_action_3 = random.randrange(0, configuration.signs)
            var_array.append(last_react_action_3)

    if var_array[observation.step] == var_array[observation.step - 1] == 0:
        last_react_action_3 = random.randrange(0, configuration.signs)
        var_array.append(last_react_action_3)
    elif var_array[observation.step] == 0:
        var_array.append(0)
    else:
        last_react_action_3 = random.randrange(0, configuration.signs)
        var_array.append(last_react_action_3)

    return var_array[observation.step]


# 12.Полный рандом
def all_random(observation, configuration):
    return random.randrange(0, configuration.signs)


agents = {
    'always_rock': always_rock,
    'always_paper': always_paper,
    'always_scissors': always_scissors,
    'copy_opponent': copy_opponent,
    'never_rock': never_rock,
    'never_paper': never_paper,
    'never_scissors': never_scissors,
    'react_if_lose': react_if_lose,
    'every_third_round': every_third_round,
    'start_with_rock': start_with_rock,
    'move_with_a_rock_two_times': move_with_a_rock_two_times,
    'all_random': all_random
}

In [5]:
from your_agent import agents as all_agents

In [6]:
# Создание словаря агентов
competition_result = {}
for each_agent in agents:
    competition_result[each_agent] = 0

print(f'{color.BOLD}РЕЗУЛЬТАТЫ{color.END}')
print()

# Соревнования каждый с каждым
number_competitions = 3
for _ in range(number_competitions):
    iteration = -1

    for agent_1 in agents.values():
        iteration += 1

        for agent_2 in dict(list(agents.items())[iteration:]).values():
            if agent_1 != agent_2:
                result = list(map(int, evaluate("rps", [agent_1, agent_2], configuration={"episodeSteps": 100})[0]))

                agent_1_name = agent_1.__name__
                agent_2_name = agent_2.__name__

                competition_result[agent_1_name] += result[0]
                competition_result[agent_2_name] += result[1]
                
                # Не выводить результаты, если счёт игры получился 0-0
                if result[0] == result[1] == 0:
                    pass
                else:
                    if result[0] > result[1]:
                        print(f'{agent_1_name}\n'
                              f'{color.GREEN}{result[0]}{color.END}')
                        print(f'{agent_2_name}\n'
                              f'{color.RED}{result[1]}{color.END} \n')
                    else:
                        print(f'{agent_1_name}\n'
                              f'{color.RED}{result[0]}{color.END}')
                        print(f'{agent_2_name}\n'
                              f'{color.GREEN}{result[1]}{color.END} \n')

РЕЗУЛЬТАТЫ

always_rock
-99
always_paper
99 

always_rock
99
always_scissors
-99 

always_rock
49
never_paper
-49 

always_rock
-53
never_scissors
53 

always_rock
-99
react_if_lose
99 

always_paper
-99
always_scissors
99 

always_paper
-46
never_rock
46 

always_paper
49
never_scissors
-49 

always_paper
-97
react_if_lose
97 

always_scissors
55
never_rock
-55 

always_scissors
-50
never_paper
50 

always_scissors
-98
react_if_lose
98 

copy_opponent
99
react_if_lose
-99 

never_rock
-35
never_paper
35 

never_rock
23
never_scissors
-23 

never_rock
-20
react_if_lose
20 

never_scissors
-27
react_if_lose
27 

react_if_lose
71
every_third_round
-71 

always_rock
-99
always_paper
99 

always_rock
99
always_scissors
-99 

always_rock
-23
never_rock
23 

always_rock
54
never_paper
-54 

always_rock
-48
never_scissors
48 

always_rock
-98
react_if_lose
98 

always_paper
-99
always_scissors
99 

always_paper
-51
never_rock
51 

always_paper
55
never_scissors
-55 

always_paper
-98
react_if

In [7]:
sorted_competition_result = {}
sorted_agents = sorted(competition_result, key=competition_result.get, reverse=True)

for agent in sorted_agents:
    sorted_competition_result[agent] = competition_result[agent]

column_numbers = [i for i in range(1, len(sorted_competition_result) + 1)]
number = iter(column_numbers)

print(color.BOLD + 'ТУРНИРНАЯ ТАБЛИЦА' + color.END)

for agent, result in sorted_competition_result.items():
    # Раскраска результатов в зависимости от того, положительный результат или нет
    if result > 0:
        print(f'{str(next(number)).rjust(3)}. {agent.ljust(22)} - {color.GREEN}{str(result).rjust(4)}{color.END}')
    elif result == 0:
        print(f'{str(next(number)).rjust(3)}. {agent.ljust(22)} - {color.BLUE}{str(result).rjust(4)}{color.END}')
    else:
        print(f'{str(next(number)).rjust(3)}. {agent.ljust(22)} - {color.RED}{str(result).rjust(4)}{color.END}')

ТУРНИРНАЯ ТАБЛИЦА
  1. react_if_lose          - 1093
  2. copy_opponent          -  149
  3. start_with_rock        -    0
  4. all_random             -    0
  5. never_paper            -  -16
  6. move_with_a_rock_two_times -  -21
  7. never_scissors         -  -40
  8. never_rock             -  -82
  9. every_third_round      - -171
 10. always_paper           - -293
 11. always_scissors        - -305
 12. always_rock            - -314
